In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 15.6 MB/s eta 0:00:00


In [2]:
import os
import glob
import cv2
import pandas as pd
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


In [3]:
# Step 2: Paths
base_dir = '/content/drive/My Drive/CV'
candidate_gt_path = os.path.join(base_dir, 'Candidate_Set', 'Candidate_Groundtruth.xlsx')
query_gt_path = os.path.join(base_dir, 'Query_Set', 'query_groundtruth.xlsx')
candidate_image_dir = os.path.join(base_dir, 'Candidate_Set', 'Candidate_Images_Selected_500')
query_image_dir = os.path.join(base_dir, 'Query_Set', 'Query_Images')

# New dataset structure
output_dir = '/content/drive/My Drive/CV Final/Dataset_augment'
os.makedirs(output_dir, exist_ok=True)
image_train_dir = os.path.join(output_dir, 'images', 'train')
image_val_dir = os.path.join(output_dir, 'images', 'val')
label_train_dir = os.path.join(output_dir, 'labels', 'train')
label_val_dir = os.path.join(output_dir, 'labels', 'val')
for path in [image_train_dir, image_val_dir, label_train_dir, label_val_dir]:
    os.makedirs(path, exist_ok=True)


In [4]:
# Step 3: Load datasets
candidate_gt = pd.read_excel(candidate_gt_path)
query_gt = pd.read_excel(query_gt_path)


In [5]:
# Helper Function to Remove White Pixels and Create Accurate Masks
def remove_white_pixels(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
    mask = cv2.bitwise_and(img, img, mask=thresh)
    return mask

# Generate Bounding Boxes Based on Masks
def generate_bounding_boxes(mask):
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = [cv2.boundingRect(c) for c in contours]  # (x, y, w, h)
    return boxes

In [30]:
def map_top_classes(df_list, image_dir, top_n=30):
    # Step 1: Count images for each class
    class_image_counts = {}

    # Iterate through all files in the candidate image directory
    for filename in os.listdir(image_dir):
        # Extract class ID from filename (assuming filenames follow the format <ProductID>_<anything>.jpg)
        class_id = filename.split('_')[0]
        class_image_counts[class_id] = class_image_counts.get(class_id, 0) + 1

    # Sort classes by image count in descending order and select top N
    top_classes = sorted(class_image_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_class_ids = {cls for cls, _ in top_classes}

    # Step 2: Create class mapping for top N classes
    class_mapping = {cls: idx for idx, cls in enumerate(sorted(top_class_ids))}

    for df in df_list:
      # Convert 'Product ID' to string with leading zeros
      df['Product ID'] = df['Product ID'].astype(str).str.zfill(7)

      # Apply the mapping to the 'Product ID' column
      df['MappedProductID'] = df['Product ID'].apply(lambda x: class_mapping.get(x, -1))  # Map to -1 if not in top classes

      #  Keep rows with valid mappings only (not -1)
      df = df[df['MappedProductID'] != -1]

      # Return the class mapping and the filtered DataFrame
    return class_mapping, df



In [31]:
class_mapping, df1 = map_top_classes([candidate_gt, query_gt], candidate_image_dir, top_n=30)
print("Top 30 Class Mapping:", class_mapping)

Top 30 Class Mapping: {'0000133': 0, '0000919': 1, '0022236': 2, '0025619': 3, '0025684': 4, '0025700': 5, '0025833': 6, '0025858': 7, '0025965': 8, '0026062': 9, '0028852': 10, '0039461': 11, '0039487': 12, '0039545': 13, '0357780': 14, '0358515': 15, '0523050': 16, '0984765': 17, '1237684': 18, '2151785': 19, '2393767': 20, '2619351': 21, '2619369': 22, '2619377': 23, '2685410': 24, '3013703': 25, '3300779': 26, '3300795': 27, '3300829': 28, '3302965': 29}


In [33]:
df1

,Query Image ID,Product ID,Product Description,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,MappedProductID
37,0010_1.jpg,0025684,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΚΕΡΑΣΙ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2547776,1,0.477064,2547776,1,0.471002,3593373,1,0.465140,4
38,0010_2.jpg,0025684,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΚΕΡΑΣΙ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2316024,0,0.496828,3098217,0,0.496463,3610482,0,0.496463,4
39,0010_3.jpg,0025684,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΚΕΡΑΣΙ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3122132,0,0.430604,2729473,0,0.430305,3335809,0,0.428907,4
40,0010_4.jpg,0025684,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΚΕΡΑΣΙ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3122132,0,0.430604,2729473,0,0.430305,3335809,0,0.428907,4
41,0011_1.jpg,0025700,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΑΝΑΝΑΣ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2193746,0,0.404662,2654432,0,0.400488,3171592,0,0.400040,5
42,0011_2.jpg,0025700,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΑΝΑΝΑΣ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55103,0,0.531435,46474,0,0.528726,3384229,0,0.527980,5
43,0011_3.jpg,0025700,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΑΝΑΝΑΣ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2729473,0,0.466367,3101953,0,0.466091,3356037,0,0.465431,5
44,0011_4.jpg,0025700,ΦΡΟΥΙ ΖΕΛΕ ΓΙΩΤΗΣ ΑΝΑΝΑΣ 200ΓΡ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2729473,0,0.466367,3101953,0,0.466091,3356037,0,0.465431,5
45,0012_1.jpg,0026062,ΑΛΕΥΡΙ ΜΑΝΝΑ ΚΙΤΡΙΝΟ 1ΚΙΛΟ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2376234,0,0.554024,2692622,0,0.552113,2719409,0,0.551176,9
46,0012_2.jpg,0026062,ΑΛΕΥΡΙ ΜΑΝΝΑ ΚΙΤΡΙΝΟ 1ΚΙΛΟ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2417038,0,0.536252,3207263,0,0.535870,2278851,0,0.535653,9


In [34]:
# Step 5: Custom Data Augmentation Functions
def horizontal_flip(image):
    return cv2.flip(image, 1)

def rotate_image(image, angle):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))

def blur_image(image, ksize=5):
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

def random_crop(image, crop_size=(400, 400)):
    h, w = image.shape[:2]
    crop_h, crop_w = crop_size
    if h < crop_h or w < crop_w:
        return image  # Return original if crop size is larger
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    return image[y:y + crop_h, x:x + crop_w]

def shear_image(image, shear_factor):
    h, w = image.shape[:2]
    M = np.array([[1, shear_factor, 0], [0, 1, 0]], dtype=np.float32)
    return cv2.warpAffine(image, M, (w, h))

def to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def adjust_brightness_contrast(image, alpha=1.0, beta=0):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def add_gaussian_noise(image, mean=0, std=10):
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    return cv2.add(image, noise)

def coarse_dropout(image, max_holes=5, max_size=50):
    h, w = image.shape[:2]
    for _ in range(max_holes):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        x1 = np.clip(x - max_size // 2, 0, w)
        x2 = np.clip(x + max_size // 2, 0, w)
        y1 = np.clip(y - max_size // 2, 0, h)
        y2 = np.clip(y + max_size // 2, 0, h)
        image[y1:y2, x1:x2] = 0  # Set region to black
    return image

# List of augmentations
augmentations = [
    horizontal_flip,
    lambda img: rotate_image(img, angle=30),
    lambda img: blur_image(img, ksize=5),
    lambda img: random_crop(img, crop_size=(400, 400)),
    lambda img: shear_image(img, shear_factor=0.2),
    to_gray,
    lambda img: adjust_brightness_contrast(img, alpha=1.2, beta=30),
    lambda img: add_gaussian_noise(img, mean=0, std=20),
    lambda img: coarse_dropout(img, max_holes=5, max_size=30),
]

In [35]:
def create_yolo_annotations_with_masks(
    df, image_dir, output_image_dir, output_label_dir, class_mapping, augment=True
):
    """
    Process images, create YOLO annotations, and optionally augment images.
    Args:
        df: DataFrame containing the product and mapped class IDs.
        image_dir: Directory containing images to process.
        output_image_dir: Directory to save processed images.
        output_label_dir: Directory to save YOLO labels.
        class_mapping: Dictionary mapping selected classes to their mapped IDs.
        augment: Boolean to enable augmentations.
    """
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for _, row in df.iterrows():
        product_id = str(row['Product ID']).zfill(7)  # Ensure zero-padding for comparison
        class_id = row['MappedProductID']
        print(product_id,class_id )

        # Skip if class_id is not in the selected top 30
        if class_id not in class_mapping.values():
            continue

        # Get all images for the current product ID
        image_pattern = os.path.join(image_dir, f"{product_id}_*.jpg")
        image_files = glob.glob(image_pattern)
        if not image_files:
            print(f"No images found for Product ID: {product_id}")
            continue

        for img_path in image_files:
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Resize the image to 640x640
            img = cv2.resize(img, (640, 640))

            # Remove white pixels and create a mask
            mask = remove_white_pixels(img)

            # Generate bounding boxes from the mask
            boxes = generate_bounding_boxes(mask)

            # Skip if no bounding boxes were generated
            if not boxes:
                continue

            # Save the original image
            image_name = os.path.basename(img_path)
            output_img_path = os.path.join(output_image_dir, image_name)
            cv2.imwrite(output_img_path, img)

            # Create a corresponding YOLO label file
            label_name = os.path.splitext(image_name)[0] + '.txt'
            label_path = os.path.join(output_label_dir, label_name)
            with open(label_path, 'w') as label_file:
                for box in boxes:
                    x, y, w, h = box
                    x_center = (x + w / 2) / 640
                    y_center = (y + h / 2) / 640
                    width = w / 640
                    height = h / 640
                    label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

            # Apply augmentations if enabled
            if augment:
                for i, aug in enumerate(augmentations):
                    try:
                        aug_img = aug(img)  # Apply augmentation
                        aug_img_name = f"{os.path.splitext(image_name)[0]}_aug_{i}.jpg"
                        aug_img_path = os.path.join(output_image_dir, aug_img_name)
                        cv2.imwrite(aug_img_path, aug_img)

                        # Create a corresponding YOLO label file for the augmented image
                        aug_label_name = f"{os.path.splitext(image_name)[0]}_aug_{i}.txt"
                        aug_label_path = os.path.join(output_label_dir, aug_label_name)
                        with open(aug_label_path, 'w') as aug_label_file:
                            for box in boxes:
                                x, y, w, h = box
                                x_center = (x + w / 2) / 640
                                y_center = (y + h / 2) / 640
                                width = w / 640
                                height = h / 640
                                aug_label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
                    except Exception as e:
                        print(f"Augmentation failed for {image_name} with error: {e}")

# Process candidate and query datasets
create_yolo_annotations_with_masks(
    candidate_gt, candidate_image_dir, image_train_dir, label_train_dir, class_mapping
)
create_yolo_annotations_with_masks(
    query_gt, query_image_dir, image_val_dir, label_val_dir, class_mapping
)


Streaming output truncated to the last 5000 lines.
2484152 -1
2484541 -1
2484566 -1
2484574 -1
2484764 -1
2486199 -1
2486306 -1
2486611 -1
2486918 -1
2487312 -1
2487965 -1
2489672 -1
2489862 -1
2490134 -1
2490894 -1
2490902 -1
2491181 -1
2491272 -1
2491280 -1
2491330 -1
2491355 -1
2491686 -1
2491710 -1
2491777 -1
2492171 -1
2492619 -1
2493187 -1
2493310 -1
2494540 -1
2494581 -1
2494607 -1
2494615 -1
2495380 -1
2495398 -1
2495778 -1
2496438 -1
2496529 -1
2497121 -1
2497543 -1
2497568 -1
2497576 -1
2498491 -1
2498871 -1
2498889 -1
2499655 -1
2499705 -1
2499820 -1
2500015 -1
2500452 -1
2500502 -1
2500510 -1
2502532 -1
2502714 -1
2503217 -1
2503845 -1
2505303 -1
2505444 -1
2507838 -1
2509305 -1
2509768 -1
2512143 -1
2512655 -1
2512887 -1
2513018 -1
2513109 -1
2513190 -1
2513273 -1
2513331 -1
2513711 -1
2513745 -1
2513752 -1
2513760 -1
2515054 -1
2516102 -1
2516110 -1
2516623 -1
2516847 -1
2517241 -1
2517282 -1
2517316 -1
2517332 -1
2517340 -1
2517357 -1
2517464 -1
2517498 -1
2517548 -1
251

In [21]:
candidate_gt

,Product ID,Description,Specific Category (SPC),Broad Category (BPC),Brand (BC),MappedProductID
0,18,ΕΛΑΙΟΛΑΔΟ ΑΛΤΙΣ 1L,Ελαιόλαδα,ΛΑΔΙ,ΑΛΤΙΣ,-1
1,59,ΕΛΑΙΟΛΑΔΟ ΜΙΝΕΡΒΑ 1 ΛΙΤ ΦΙΑΛΗ,Ελαιόλαδα,ΛΑΔΙ,ΜΙΝΕΡΒΑ,-1
2,67,ΕΛΑΙΟΛΑΔΟ ΧΩΡΙΟ ΚΟΡΩΝΕΙΚΗ ΠΟΙΚΙΛΙΑ 1L,Ελαιόλαδα,ΛΑΔΙ,ΧΩΡΙΟ,-1
3,83,ΕΛΑΙΟΛΑΔΟ NISSA 1ΛΙΤ S 01 ΦΙΑΛΗ,Ελαιόλαδα,ΛΑΔΙ,NISSA ΕΛΑΙΟΛΑΔΑ,-1
4,133,ΕΛΑΙΟΛΑΔΟ ΜΙΝΕΡΒΑ 2L,Ελαιόλαδα,ΛΑΔΙ,ΜΙΝΕΡΒΑ,-1
...,...,...,...,...,...,...
6343,984,ΗΛΙΕΛΑΙΟ SANOLA 2L,Ηλιέλαια,ΛΑΔΙ,SANOLA ΣΠΟΡΕΛΑΙΑ,-1
6344,985176,BECEL PRO ACTIV 250ΓΡ,Μαργαρίνες λειτουργικές,ΜΑΡΓΑΡΙΝΕΣ,BECEL,-1
6345,994616,ΣΕΡΒ/ΚΙΑ EV DAY ALL COT LARGE 30Τ,Σερβιετάκια,ΣΕΡΒΙΕΤΑΚΙΑ,EVERY DAY,-1
6346,9951,"ΚΡΑΣΙ ΕΡΥΘΡΟ ΜΑΚΕΔΟΝΙΚΟ ΤΣΑΝΤ 0,75L",Κρασιά κόκκινα,ΚΡΑΣΙΑ,ΤΣΑΝΤΑΛΗ,-1


In [16]:
print(class_mapping.values())

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])


In [36]:
import shutil
from sklearn.model_selection import train_test_split
import glob
import os

# Step 1: Filter images and labels for the selected top 30 classes (including original and augmented)
def filter_images_by_classes(image_dir, label_dir, class_mapping):
    # Get all images in the directory (both original and augmented)
    image_files = glob.glob(f"{image_dir}/*.jpg")
    filtered_images = []

    for img_path in image_files:
        image_name = os.path.basename(img_path)
        label_path = os.path.splitext(img_path.replace(image_dir, label_dir))[0] + '.txt'

        # Extract the product ID from the image name (Assuming it's mapped from the image name)
        product_id = image_name.split('_')[0]  # Get the first part of the image name (product ID)

        # Check if the product ID is in the top 30 class mapping
        if product_id in class_mapping:
            filtered_images.append((img_path, label_path))

    return filtered_images

# Step 2: Split the filtered images into train and validation sets
def split_and_move_files(filtered_images, image_train_dir, label_train_dir, image_val_dir, label_val_dir):
    # Split the filtered images into train and validation sets (80% train, 20% validation)
    train_files, val_files = train_test_split(filtered_images, test_size=0.2, random_state=42)

    # Move validation images and labels to the validation folder
    for img_path, label_path in val_files:
        # Move image
        val_file_name = os.path.basename(img_path)
        dest_image_path = os.path.join(image_val_dir, val_file_name)

        if os.path.exists(dest_image_path):
            base_name, ext = os.path.splitext(val_file_name)
            dest_image_path = os.path.join(image_val_dir, f"{base_name}_conflict{ext}")

        shutil.move(img_path, dest_image_path)

        # Move corresponding label file
        label_file_name = os.path.basename(label_path)
        dest_label_path = os.path.join(label_val_dir, label_file_name)

        if os.path.exists(dest_label_path):
            base_name, ext = os.path.splitext(label_file_name)
            dest_label_path = os.path.join(label_val_dir, f"{base_name}_conflict{ext}")

        shutil.move(label_path, dest_label_path)

    # Move training images and labels to the training folder
    for img_path, label_path in train_files:
        # Move image
        train_file_name = os.path.basename(img_path)
        dest_image_path = os.path.join(image_train_dir, train_file_name)

        if os.path.exists(dest_image_path):
            base_name, ext = os.path.splitext(train_file_name)
            dest_image_path = os.path.join(image_train_dir, f"{base_name}_conflict{ext}")

        shutil.move(img_path, dest_image_path)

        # Move corresponding label file
        label_file_name = os.path.basename(label_path)
        dest_label_path = os.path.join(label_train_dir, label_file_name)

        if os.path.exists(dest_label_path):
            base_name, ext = os.path.splitext(label_file_name)
            dest_label_path = os.path.join(label_train_dir, f"{base_name}_conflict{ext}")

        shutil.move(label_path, dest_label_path)

# Step 3: Execute the full process
# Filter both original and augmented images for top 30 classes
filtered_images = filter_images_by_classes(image_train_dir, label_train_dir, class_mapping)

# Split and move files into train and validation sets
split_and_move_files(filtered_images, image_train_dir, label_train_dir, image_val_dir, label_val_dir)


In [37]:
import yaml
import os

# Assuming 'class_mapping' is a dictionary mapping product IDs to class indices.
# For example: {'0000133': 0, '0000919': 1, ...}

# Convert class_mapping to ensure keys and values are strings
class_mapping_str = {str(k): str(v) for k, v in class_mapping.items()}

# Define the path for the YOLO data configuration YAML file
yolo_data_config_path = os.path.join(base_dir, 'yolo_data_config.yaml')

# Prepare the data configuration dictionary
data_config = {
    'train': image_train_dir,  # Path to the training images directory
    'val': image_val_dir,      # Path to the validation images directory
    'nc': len(class_mapping_str),  # Number of classes
    'names': list(class_mapping_str.keys())  # List of class names (product IDs) as strings
}

# Save the configuration to the YAML file
with open(yolo_data_config_path, 'w') as file:
    yaml.dump(data_config, file, default_flow_style=False)

# Print the resulting configuration for verification (optional)
print(f"YOLO data configuration saved to: {yolo_data_config_path}")


YOLO data configuration saved to: /content/drive/My Drive/CV/yolo_data_config.yaml


In [39]:
# Import the necessary libraries
from ultralytics import YOLO

# Define the model and training parameters
model1 = YOLO('yolov8n.pt')  # Load the YOLOv8 model (nano version)

# Start training with custom data configuration
model1.train(
    data=yolo_data_config_path,  # Path to the YOLO data config file
    epochs=10,  # Number of epochs for training
    imgsz=256,  # Image size (resolution)
    batch=8,    # Batch size
    cache=False,  # Avoid caching the data to disk (set to True if necessary)
    save_dir='/content/runs/detect/train4',  # Directory where the trained models and results will be saved
)

# Check if training has completed
print("Training completed! Check results in:", '/content/runs/detect/train4')


Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/CV/yolo_data_config.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 14.1MB/s]


Overriding model.yaml nc=80 with nc=30

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/My Drive/CV Final/Dataset_augment/labels/train... 1120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1120/1120 [00:22<00:00, 49.42it/s]


train: New cache created: /content/drive/My Drive/CV Final/Dataset_augment/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      10/10         0G     0.5944      1.446      1.027         26        256: 100%|██████████| 140/140 [02:55<00:00,  1.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:22<00:00,  1.25s/it]

                   all        280       2027      0.627      0.428      0.427      0.394



10 epochs completed in 0.564 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,011,498 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:18<00:00,  1.04s/it]


                   all        280       2027      0.622      0.433      0.427      0.394
               0000133          5          7      0.757      0.714      0.717      0.681
               0000919         12         15        0.9        0.8      0.794      0.747
               0022236          9         58      0.528      0.155      0.153      0.149
               0025619          4        110       0.26     0.0364     0.0237     0.0209
               0025684          8         50      0.265       0.06     0.0689     0.0621
               0025700         13         17      0.714      0.765      0.764      0.675
               0025833         10        134      0.729     0.0672       0.09     0.0855
               0025858         11         57      0.703      0.175      0.186      0.164
               0025965          5         10      0.665        0.5      0.499      0.475
               0026062         10         51       0.86      0.196      0.207      0.193
               002885

In [42]:
from ultralytics import YOLO

# Load the trained model (best.pt)
m = YOLO('/content/drive/My Drive/CV Final/best.pt')
# Perform validation
metrics = m.val()


Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,011,498 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/My Drive/CV Final/Dataset_augment/labels/val.cache... 280 images, 0 backgrounds, 0 corrupt: 100%|██████████| 280/280 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:20<00:00,  1.13s/it]


                   all        280       2027      0.622      0.433      0.427      0.394
               0000133          5          7      0.757      0.714      0.717      0.681
               0000919         12         15        0.9        0.8      0.794      0.747
               0022236          9         58      0.528      0.155      0.153      0.149
               0025619          4        110       0.26     0.0364     0.0237     0.0209
               0025684          8         50      0.265       0.06     0.0689     0.0621
               0025700         13         17      0.714      0.765      0.764      0.675
               0025833         10        134      0.729     0.0672       0.09     0.0855
               0025858         11         57      0.703      0.175      0.186      0.164
               0025965          5         10      0.665        0.5      0.499      0.475
               0026062         10         51       0.86      0.196      0.207      0.193
               002885

In [43]:
# Access and display the key metrics
print("\nValidation Metrics:")
print(f"mAP@0.5: {metrics.box.map50:.4f}")  # mAP at IoU=0.5
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")  # mAP averaged over IoU=0.5 to 0.95
print(f"Mean Precision: {metrics.box.mp:.4f}")  # Mean Precision
print(f"Mean Recall: {metrics.box.mr:.4f}")  # Mean Recall


Validation Metrics:
mAP@0.5: 0.4270
mAP@0.5:0.95: 0.3944
Mean Precision: 0.6220
Mean Recall: 0.4329
